In [2]:
# import liabraries
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime
import isodate

In [ ]:
# log collection datetime
trending_date = datetime.now()
file_date = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
file_date

'2025-11-18_13-28-58'

In [ ]:
# ========================
# STEP 1: ENTER YOUR API KEY
# ========================
API_KEY = 'xxxxxxx'
BASE_URL = "https://www.googleapis.com/youtube/v3"

In [31]:
# ========================
# STEP 2: DEFINE SEARCH PARAMETERS
# ========================

# parameters for United States and Canada
region_list = ['US', 'CA']
params = {}
for region in region_list:
    params[region] = {
        'part': 'snippet, contentDetails, statistics, topicDetails, status',
        'chart': 'mostPopular',
        'maxResults': 50,
        'regionCode': region, # Change region code as needed
        'key': API_KEY
        }

# # parameters for Canada
# params_ca = {
#     'part': 'snippet, contentDetails, statistics, topicDetails',
#     'chart': 'mostPopular',
#     'maxResults': 50,
#     'regionCode': 'CA', # Change region code as needed
#     'key': API_KEY
#     }

In [32]:
# ========================
# STEP 3: CALL YOUTUBE API
# ========================
data = {}
for region in region_list:
    response = requests.get(f"{BASE_URL}/videos", params=params[region])
    if response.status_code == 200:
        data[region] = response.json()
    else:
        print("Error:", response.status_code, response.text)
        data = None

In [ ]:
# ========================
# STEP 4: PARSE DATA INTO DATAFRAME
# ========================
video_list = []
for region in region_list:
    if data[region]:
        for i, item in enumerate(data[region].get('items', [])):
            video = {
                'trending_date': trending_date,
                'trending_country': region,
                'rank': i+1,
                'video_id': item['id'],
                'video_title': item['snippet']['title'],
                'video_published_at': item['snippet']['publishedAt'],
                'video_description': item['snippet'].get('description',''),
                'channel_id': item['snippet']['channelId'],
                'channel_title': item['snippet']['channelTitle'],
                'tags': item['snippet'].get('tags', []),
                'category_id': int(item['snippet']['categoryId']),
                'default_language': item['snippet'].get('defaultLanguage', ''),
                # 'default_audio_language': item['snippet'].get('defaultAudioLanguage', ''),
                'duration': item['contentDetails']['duration'],
                'dimension': item['contentDetails']['dimension'],
                'definition': item['contentDetails']['definition'],
                'licensed_content': item['contentDetails']['licensedContent'],

                'video_view_count': int(item['statistics'].get('viewCount', 0)) if 'viewCount' in item['statistics'] else np.nan,
                'video_like_count': int(item['statistics'].get('likeCount', 0)) if 'likeCount' in item['statistics'] else np.nan,
                # 'video_dislike_count': int(item['statistics'].get('dislikeCount', 0)) if 'dislikeCount' in item['statistics'] else np.nan,
                # 'video_favorite_count': int(item['statistics']['favoriteCount']) if 'favoriteCount' in item['statistics'] else np.nan,
                'video_comment_count': int(item['statistics'].get('commentCount', 0)) if 'commentCount' in item['statistics'] else np.nan,

                # 'topic_id': item['topicDetails'].get('topicIds', ''),
                # 'relevant_topic_id': item['topicDetails'].get('relevantTopicIds', ''),
                'topic_categories': item.get('topicDetails', {}).get('topicCategories', ''),
                
                'made_for_kids': item['status']['madeForKids'],
                # 'width_pixels': item['fileDetails']['videoStreams']['widthPixels'],
                # 'height_pixels': item['fileDetails']['videoStreams']['heightPixels'],
                # 'frame_rate_fps': item['fileDetails']['videoStreams']['frameRateFps'],
                # 'codec': item['fileDetails']['videoStreams']['codec'],
                # 'bitrate_bps': item['fileDetails']['videoStreams']['bitrateBps'],
                # 'frame_rate_fps': item['fileDetails']['videoStreams']['frameRateFps'],
                }
            video_list.append(video)

        dfv = pd.DataFrame(video_list)
        display(dfv.head())
        file = f'youtube_trending_videos_{file_date}.csv'
        dfv.to_csv(f'youtube_trending_videos_{file_date}.csv', index=False)
        print(f" Saved trending videos to youtube_trending_videos_{file_date}.csv")
    else:
        print("No data fetched.")

,trending_date,trending_country,rank,video_id,video_title,video_published_at,video_description,channel_id,channel_title,tags,...,default_language,duration,dimension,definition,licensed_content,video_view_count,video_like_count,video_comment_count,topic_categories,made_for_kids
0,2025-11-18 13:28:58.468121,US,1,ySJE71dxJoA,Fine Chiki,2025-11-07T06:58:26Z,Provided to YouTube by DistroKid\n\nFine Chiki...,UCb9z1Q2JHslU7mB3czuN7XQ,JNR VIGI - Topic,"[JNR VIGI, Arah Ola, Emmanuel Ola, Jacob Hines...",...,en,PT3M42S,2d,hd,True,81446,2610.0,78,"[https://en.wikipedia.org/wiki/Hip_hop_music, ...",False
1,2025-11-18 13:28:58.468121,US,2,QKYFfYLe5rs,Moana | Official Teaser,2025-11-17T17:00:47Z,"#Moana, arriving only in theaters July 10, 202...",UC_5niPa-d35gg88HaS7RrIw,Disney,[],...,en,PT1M,2d,hd,True,4035943,31846.0,8151,"[https://en.wikipedia.org/wiki/Entertainment, ...",False
2,2025-11-18 13:28:58.468121,US,3,NxISG05CANY,$0 - $1 Trillion Only FISHING in Steal a Brain...,2025-11-17T22:57:14Z,🎮 I Released My Own ROBLOX GAME! (Dump)\nhttps...,UCsPMNatXCwqyUuJilBT73jg,CaylusBlox,"[caylusblox, caylus roblox, caylus, caylus gam...",...,en-US,PT18M2S,2d,hd,True,2150292,27679.0,2555,[https://en.wikipedia.org/wiki/Action-adventur...,False
3,2025-11-18 13:28:58.468121,US,4,tG1HKY6Jwas,Charli xcx - Chains of Love (Official Video),2025-11-17T17:00:56Z,Charli xcx - Chains of Love\nStream: https://c...,UCOvEMf00DnwdkpOCRWdc6uA,Charli xcx,"[charli xcx, charli xcx wuthering heights, wut...",...,en,PT3M13S,2d,hd,True,458239,38006.0,1961,[https://en.wikipedia.org/wiki/Independent_mus...,False
4,2025-11-18 13:28:58.468121,US,5,BKOVzHcjEIo,Dhurandhar Official Trailer | Ranveer Singh | ...,2025-11-18T06:50:49Z,Tum Logon Ke Patakhe Khatm Hogaye Ho Toh Mai D...,UCcXQd6kHKm0b41x8zMVMmMg,JioStudios,"[Jio Studios, Dhurandhar First Look, dhurandha...",...,en,PT4M8S,2d,hd,False,4174285,220865.0,16063,"[https://en.wikipedia.org/wiki/Entertainment, ...",False


 Saved trending videos to youtube_trending_videos_2025-11-18_13-28-58.csv


,trending_date,trending_country,rank,video_id,video_title,video_published_at,video_description,channel_id,channel_title,tags,...,default_language,duration,dimension,definition,licensed_content,video_view_count,video_like_count,video_comment_count,topic_categories,made_for_kids
0,2025-11-18 13:28:58.468121,US,1,ySJE71dxJoA,Fine Chiki,2025-11-07T06:58:26Z,Provided to YouTube by DistroKid\n\nFine Chiki...,UCb9z1Q2JHslU7mB3czuN7XQ,JNR VIGI - Topic,"[JNR VIGI, Arah Ola, Emmanuel Ola, Jacob Hines...",...,en,PT3M42S,2d,hd,True,81446,2610.0,78,"[https://en.wikipedia.org/wiki/Hip_hop_music, ...",False
1,2025-11-18 13:28:58.468121,US,2,QKYFfYLe5rs,Moana | Official Teaser,2025-11-17T17:00:47Z,"#Moana, arriving only in theaters July 10, 202...",UC_5niPa-d35gg88HaS7RrIw,Disney,[],...,en,PT1M,2d,hd,True,4035943,31846.0,8151,"[https://en.wikipedia.org/wiki/Entertainment, ...",False
2,2025-11-18 13:28:58.468121,US,3,NxISG05CANY,$0 - $1 Trillion Only FISHING in Steal a Brain...,2025-11-17T22:57:14Z,🎮 I Released My Own ROBLOX GAME! (Dump)\nhttps...,UCsPMNatXCwqyUuJilBT73jg,CaylusBlox,"[caylusblox, caylus roblox, caylus, caylus gam...",...,en-US,PT18M2S,2d,hd,True,2150292,27679.0,2555,[https://en.wikipedia.org/wiki/Action-adventur...,False
3,2025-11-18 13:28:58.468121,US,4,tG1HKY6Jwas,Charli xcx - Chains of Love (Official Video),2025-11-17T17:00:56Z,Charli xcx - Chains of Love\nStream: https://c...,UCOvEMf00DnwdkpOCRWdc6uA,Charli xcx,"[charli xcx, charli xcx wuthering heights, wut...",...,en,PT3M13S,2d,hd,True,458239,38006.0,1961,[https://en.wikipedia.org/wiki/Independent_mus...,False
4,2025-11-18 13:28:58.468121,US,5,BKOVzHcjEIo,Dhurandhar Official Trailer | Ranveer Singh | ...,2025-11-18T06:50:49Z,Tum Logon Ke Patakhe Khatm Hogaye Ho Toh Mai D...,UCcXQd6kHKm0b41x8zMVMmMg,JioStudios,"[Jio Studios, Dhurandhar First Look, dhurandha...",...,en,PT4M8S,2d,hd,False,4174285,220865.0,16063,"[https://en.wikipedia.org/wiki/Entertainment, ...",False


 Saved trending videos to youtube_trending_videos_2025-11-18_13-28-58.csv


In [8]:
dfv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   trending_date        100 non-null    datetime64[ns]
 1   trending_country     100 non-null    object        
 2   rank                 100 non-null    int64         
 3   video_id             100 non-null    object        
 4   video_title          100 non-null    object        
 5   video_published_at   100 non-null    object        
 6   video_description    100 non-null    object        
 7   channel_id           100 non-null    object        
 8   channel_title        100 non-null    object        
 9   tags                 100 non-null    object        
 10  category_id          100 non-null    int64         
 11  default_language     100 non-null    object        
 12  duration             100 non-null    object        
 13  dimension            100 non-null   

In [8]:
dfv.isnull().sum()

trending_date          0
trending_country       0
rank                   0
video_id               0
video_title            0
video_published_at     0
video_description      0
channel_id             0
channel_title          0
tags                   0
category_id            0
default_language       0
duration               0
dimension              0
definition             0
licensed_content       0
video_view_count       1
video_like_count       1
video_comment_count    1
topic_categories       0
made_for_kids          0
dtype: int64

In [298]:
cols = dfv.columns
cols

Index(['trending_date', 'trending_country', 'rank', 'video_id', 'video_title',
       'video_published_at', 'video_description', 'channel_id',
       'channel_title', 'tags', 'category_id', 'default_language', 'duration',
       'dimension', 'definition', 'licensed_content', 'video_view_count',
       'video_like_count', 'video_comment_count', 'topic_categories',
       'made_for_kids'],
      dtype='object')

In [ ]:
# Get unique channel IDs and split into chunks of 50
channel_id_list = dfv['channel_id'].unique()
print(len(channel_id_list))
channel_ids = {}
if len(channel_id_list)>50:
    channel_ids[0] = ','.join(channel_id_list[:50])
    channel_ids[1] = ','.join(channel_id_list[50:])
else:
    channel_ids[0] = ','.join(channel_id_list)

print(len(channel_ids))

73
2


In [35]:
# ========================
# STEP 2: DEFINE SEARCH PARAMETERS
# ========================
params_channel = {}
for i in range(len(channel_ids)):
    params_channel[i] = {
        'part': 'snippet, statistics, topicDetails',
        'id': channel_ids[i], # Change region code as needed
        'key': API_KEY
        }


In [36]:
# ========================
# STEP 3: CALL YOUTUBE API
# ========================
data = {}
for i in range(len(params_channel)):
    response = requests.get(f"{BASE_URL}/channels", params=params_channel[i])
    if response.status_code == 200:
        data[i] = response.json()
    else:
        print("Error:", response.status_code, response.text)
        data = None

In [37]:
# ========================
# STEP 4: PARSE DATA INTO DATAFRAME
# ========================
channel_list = []
for i in range(len(params_channel)):
    if data[i]:
        for item in data[i].get('items', []):
            channel = {
                'channel_id': item['id'],
                'channel_published_at': item['snippet']['publishedAt'],
                'channel_description': item['snippet'].get('description',''),
                'channel_country': item['snippet'].get('country',''),

                'channel_view_count': int(item['statistics'].get('viewCount', 0)) if 'viewCount' in item['statistics'] else np.nan,
                'subscriber_count': int(item['statistics'].get('subscriberCount', 0)) if 'subscriberCount' in item['statistics'] else np.nan,
                'video_count': int(item['statistics'].get('videoCount', 0)) if 'videoCount' in item['statistics'] else np.nan,
                }
            channel_list.append(channel)

        dfc = pd.DataFrame(channel_list)
        display(dfc.head())
        # Save results to CSV for later analysis
        filec = f'channel_{file_date}.csv'
        dfc.to_csv(f'channel_{file_date}.csv', index=False)
        print(f" Saved channels to channel_{file_date}.csv")
    else:
        print("No data fetched.")

,channel_id,channel_published_at,channel_description,channel_country,channel_view_count,subscriber_count,video_count
0,UCke6I9N4KfC968-yRcd5YRg,2009-11-28T02:52:35Z,Welcome to my channel! I upload a lot of funn...,US,16702569782,25200000,3598
1,UC5FJPD6II5WjCTbJ_LilgMQ,2016-02-26T13:49:31Z,Sean Rii is a multifaceted singer / songwriter...,,8117817,2260,114
2,UCzzF1kdBgwv5RB6rbB6kVSA,2020-03-19T22:48:34.528482Z,Daily videos with Tyler & Snowi! Subscribe so ...,US,2567507967,5240000,1139
3,UCRp--eWwsLI_uIkCnsbfwFQ,2006-09-19T06:22:01Z,Official channel of Timeless Music. Timeless M...,US,975155200,1070000,427
4,UCz-jVF7_acwSvt30vGJRTOw,2025-10-06T22:57:30.407413Z,🎶 Bienvenidos a SabroSón 🎶\n\nAquí encontrarás...,CO,1797618,460,10


 Saved channels to channel_2025-11-18_13-28-58.csv


,channel_id,channel_published_at,channel_description,channel_country,channel_view_count,subscriber_count,video_count
0,UCke6I9N4KfC968-yRcd5YRg,2009-11-28T02:52:35Z,Welcome to my channel! I upload a lot of funn...,US,16702569782,25200000,3598
1,UC5FJPD6II5WjCTbJ_LilgMQ,2016-02-26T13:49:31Z,Sean Rii is a multifaceted singer / songwriter...,,8117817,2260,114
2,UCzzF1kdBgwv5RB6rbB6kVSA,2020-03-19T22:48:34.528482Z,Daily videos with Tyler & Snowi! Subscribe so ...,US,2567507967,5240000,1139
3,UCRp--eWwsLI_uIkCnsbfwFQ,2006-09-19T06:22:01Z,Official channel of Timeless Music. Timeless M...,US,975155200,1070000,427
4,UCz-jVF7_acwSvt30vGJRTOw,2025-10-06T22:57:30.407413Z,🎶 Bienvenidos a SabroSón 🎶\n\nAquí encontrarás...,CO,1797618,460,10


 Saved channels to channel_2025-11-18_13-28-58.csv


In [161]:
dfc.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   channel_id            70 non-null     object
 1   channel_published_at  70 non-null     object
 2   channel_description   70 non-null     object
 3   channel_country       70 non-null     object
 4   channel_view_count    70 non-null     int64 
 5   subscriber_count      70 non-null     int64 
 6   video_count           70 non-null     int64 
dtypes: int64(3), object(4)
memory usage: 4.0+ KB


In [228]:
dfc.isnull().sum()

channel_id              0
channel_published_at    0
channel_description     0
channel_country         0
channel_view_count      0
subscriber_count        0
video_count             0
dtype: int64

In [ ]:
# merge video info and channel info
merged_df = pd.merge(dfv, dfc, on='channel_id', how='left')
merged_df.head()

,trending_date,trending_country,rank,video_id,video_title,video_published_at,video_description,channel_id,channel_title,tags,...,video_like_count,video_comment_count,topic_categories,made_for_kids,channel_published_at,channel_description,channel_country,channel_view_count,subscriber_count,video_count
0,2025-11-18 13:28:58.468121,US,1,ySJE71dxJoA,Fine Chiki,2025-11-07T06:58:26Z,Provided to YouTube by DistroKid\n\nFine Chiki...,UCb9z1Q2JHslU7mB3czuN7XQ,JNR VIGI - Topic,"[JNR VIGI, Arah Ola, Emmanuel Ola, Jacob Hines...",...,2610.0,78,"[https://en.wikipedia.org/wiki/Hip_hop_music, ...",False,2024-12-13T18:55:31.929269Z,"JNR VIGI, are talented singers/ songwriters fr...",,1922557,414,25
1,2025-11-18 13:28:58.468121,US,2,QKYFfYLe5rs,Moana | Official Teaser,2025-11-17T17:00:47Z,"#Moana, arriving only in theaters July 10, 202...",UC_5niPa-d35gg88HaS7RrIw,Disney,[],...,31846.0,8151,"[https://en.wikipedia.org/wiki/Entertainment, ...",False,2012-02-11T12:07:58Z,Welcome to the official Disney network on YouT...,,1118721726,6480000,1385
2,2025-11-18 13:28:58.468121,US,3,NxISG05CANY,$0 - $1 Trillion Only FISHING in Steal a Brain...,2025-11-17T22:57:14Z,🎮 I Released My Own ROBLOX GAME! (Dump)\nhttps...,UCsPMNatXCwqyUuJilBT73jg,CaylusBlox,"[caylusblox, caylus roblox, caylus, caylus gam...",...,27679.0,2555,[https://en.wikipedia.org/wiki/Action-adventur...,False,2024-08-02T07:21:37.449899Z,i play roblox here\nsubscribe and join the CAY...,US,2126321367,6830000,573
3,2025-11-18 13:28:58.468121,US,4,tG1HKY6Jwas,Charli xcx - Chains of Love (Official Video),2025-11-17T17:00:56Z,Charli xcx - Chains of Love\nStream: https://c...,UCOvEMf00DnwdkpOCRWdc6uA,Charli xcx,"[charli xcx, charli xcx wuthering heights, wut...",...,38006.0,1961,[https://en.wikipedia.org/wiki/Independent_mus...,False,2011-03-24T18:09:55Z,The official YouTube Channel of Charli xcx\n\n...,,2178980183,3960000,309
4,2025-11-18 13:28:58.468121,US,5,BKOVzHcjEIo,Dhurandhar Official Trailer | Ranveer Singh | ...,2025-11-18T06:50:49Z,Tum Logon Ke Patakhe Khatm Hogaye Ho Toh Mai D...,UCcXQd6kHKm0b41x8zMVMmMg,JioStudios,"[Jio Studios, Dhurandhar First Look, dhurandha...",...,220865.0,16063,"[https://en.wikipedia.org/wiki/Entertainment, ...",False,2018-11-05T06:07:05Z,You Are Not Ready For This!\n#Dhurandhar In Ci...,IN,921465466,1160000,1169


In [13]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   trending_date         100 non-null    datetime64[ns]
 1   trending_country      100 non-null    object        
 2   rank                  100 non-null    int64         
 3   video_id              100 non-null    object        
 4   video_title           100 non-null    object        
 5   video_published_at    100 non-null    object        
 6   video_description     100 non-null    object        
 7   channel_id            100 non-null    object        
 8   channel_title         100 non-null    object        
 9   tags                  100 non-null    object        
 10  category_id           100 non-null    int64         
 11  default_language      100 non-null    object        
 12  duration              100 non-null    object        
 13  dimension            

In [309]:
len(merged_df.columns)

27

In [39]:
merged_df.to_csv(f'org_{file_date}.csv', index=False)

In [40]:
final_file = 'youtube_trending_data.csv'
dfo = pd.read_csv(final_file, parse_dates=['trending_date'])

In [41]:
dfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4449 entries, 0 to 4448
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   trending_date         4449 non-null   datetime64[ns]
 1   trending_country      4449 non-null   object        
 2   rank                  4449 non-null   int64         
 3   video_id              4449 non-null   object        
 4   video_title           4449 non-null   object        
 5   video_published_at    4449 non-null   object        
 6   video_description     4396 non-null   object        
 7   channel_id            4449 non-null   object        
 8   channel_title         4449 non-null   object        
 9   tags                  4449 non-null   object        
 10  category_id           4449 non-null   int64         
 11  default_language      4398 non-null   object        
 12  duration              4449 non-null   object        
 13  dimension         

In [272]:
len(dfo.columns)

26

In [ ]:
# append new data to existing data
df = pd.concat([dfo, merged_df], ignore_index=True)
df.to_csv(final_file, index=False)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4549 entries, 0 to 4548
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   trending_date         4549 non-null   datetime64[ns]
 1   trending_country      4549 non-null   object        
 2   rank                  4549 non-null   int64         
 3   video_id              4549 non-null   object        
 4   video_title           4549 non-null   object        
 5   video_published_at    4549 non-null   object        
 6   video_description     4496 non-null   object        
 7   channel_id            4549 non-null   object        
 8   channel_title         4549 non-null   object        
 9   tags                  4549 non-null   object        
 10  category_id           4549 non-null   int64         
 11  default_language      4498 non-null   object        
 12  duration              4549 non-null   object        
 13  dimension         

In [169]:
df['made_for_kids'].value_counts()

made_for_kids
False    1100
Name: count, dtype: int64